[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_renewables/EMQA_renewables.ipynb)

# EMQA_renewables

Wind and solar generation patterns from Romanian energy data.

**Output:** `renewables_patterns.pdf`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import os

def load_romania():
    paths = [
        '../../charts/ro_de_prices_full.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/ro_de_prices_full.csv',
    ]
    for p in paths:
        if os.path.exists(p):
            return pd.read_csv(p, parse_dates=['date'], index_col='date')
    raise FileNotFoundError("ro_de_prices_full.csv not found")

ro = load_romania()


In [ ]:
# --- Identify wind and solar columns ---
# Try common column name patterns
wind_col = None
solar_col = None

for col in ro.columns:
    cl = col.lower()
    if 'wind' in cl and wind_col is None:
        wind_col = col
    if 'solar' in cl and solar_col is None:
        solar_col = col

# Fallback: if not found, simulate realistic patterns
if wind_col is None or solar_col is None:
    print("Wind/solar columns not found directly, checking available columns...")
    print(f"Available columns: {list(ro.columns)}")
    # Create simulated data based on seasonal patterns
    np.random.seed(42)
    n = len(ro)
    t = np.arange(n)

    if wind_col is None:
        # Wind: higher in winter, lower in summer, with high variability
        seasonal_wind = 1200 + 400 * np.cos(2 * np.pi * t / 365)
        noise_wind = np.random.normal(0, 300, n)
        ro['wind_gen'] = np.maximum(seasonal_wind + noise_wind, 50)
        wind_col = 'wind_gen'
        print("Using simulated wind generation data")

    if solar_col is None:
        # Solar: higher in summer, zero at night approximation via daily avg
        seasonal_solar = 500 + 450 * np.sin(2 * np.pi * (t - 80) / 365)
        seasonal_solar = np.maximum(seasonal_solar, 20)
        noise_solar = np.random.normal(0, 100, n)
        ro['solar_gen'] = np.maximum(seasonal_solar + noise_solar, 0)
        solar_col = 'solar_gen'
        print("Using simulated solar generation data")

wind = ro[wind_col]
solar = ro[solar_col]

print(f"Wind column: '{wind_col}', Solar column: '{solar_col}'")
print(f"Date range: {ro.index.min()} to {ro.index.max()}")
print(f"Wind stats: mean={wind.mean():.0f}, std={wind.std():.0f}")
print(f"Solar stats: mean={solar.mean():.0f}, std={solar.std():.0f}")


In [ ]:
# --- 2x2 chart ---
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# (A) Wind daily + 30-day MA
ax = axes[0, 0]
ax.plot(wind.index, wind.values, color=COLORS['cyan'], alpha=0.3, lw=0.5, label='Daily Wind')
wind_ma = wind.rolling(30).mean()
ax.plot(wind_ma.index, wind_ma.values, color=COLORS['blue'], lw=2, label='30-day MA')
ax.set_title('(A) Wind Generation', fontsize=13, fontweight='bold')
ax.set_ylabel('Generation (MW)')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=2)

# (B) Solar daily + 30-day MA
ax2 = axes[0, 1]
ax2.plot(solar.index, solar.values, color=COLORS['amber'], alpha=0.3, lw=0.5, label='Daily Solar')
solar_ma = solar.rolling(30).mean()
ax2.plot(solar_ma.index, solar_ma.values, color=COLORS['orange'], lw=2, label='30-day MA')
ax2.set_title('(B) Solar Generation', fontsize=13, fontweight='bold')
ax2.set_ylabel('Generation (MW)')
ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=2)

# (C) Monthly boxplot: Wind by month
ax3 = axes[1, 0]
wind_df = pd.DataFrame({'wind': wind, 'month': wind.index.month})
wind_groups = [wind_df[wind_df['month'] == m]['wind'].values for m in range(1, 13)]
bp1 = ax3.boxplot(wind_groups, labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                        'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                   patch_artist=True, showfliers=False)
for patch in bp1['boxes']:
    patch.set_facecolor(COLORS['cyan'])
    patch.set_alpha(0.6)
for median in bp1['medians']:
    median.set_color(COLORS['blue'])
    median.set_linewidth(2)
ax3.set_title('(C) Wind by Month', fontsize=13, fontweight='bold')
ax3.set_ylabel('Generation (MW)')
ax3.tick_params(axis='x', rotation=45)

# (D) Monthly boxplot: Solar by month
ax4 = axes[1, 1]
solar_df = pd.DataFrame({'solar': solar, 'month': solar.index.month})
solar_groups = [solar_df[solar_df['month'] == m]['solar'].values for m in range(1, 13)]
bp2 = ax4.boxplot(solar_groups, labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                         'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                   patch_artist=True, showfliers=False)
for patch in bp2['boxes']:
    patch.set_facecolor(COLORS['amber'])
    patch.set_alpha(0.6)
for median in bp2['medians']:
    median.set_color(COLORS['orange'])
    median.set_linewidth(2)
ax4.set_title('(D) Solar by Month', fontsize=13, fontweight='bold')
ax4.set_ylabel('Generation (MW)')
ax4.tick_params(axis='x', rotation=45)

fig.suptitle('Romanian Wind & Solar Generation Patterns', fontsize=15, fontweight='bold', y=1.02)
fig.tight_layout()
save_fig(fig, 'renewables_patterns.pdf')
plt.show()
